# Inverse kinematics
This new chapter considers the problem of inverse kinematics, ie solving at each control cycle of the robot a quadratic program from the derivatives (jacobian) of the current state. 


## A mobile robot for our tests

We are going to use again the UR5 robot model, however this time mounted as a mobile robot. The robot has 3+6 DOF and can move (2 translations + 1 rotation) freely on the plane. 

Two operation frames have been defined: at the front of the basis, and at the tip of the tool. 
They are displayed when the robot moves.



In [28]:
import pinocchio 
from pinocchio.utils import *
from tp3.mobilerobot import MobileRobotWrapper
pkg = 'models/'
urdf = pkg + 'ur_description/urdf/ur5_gripper.urdf'
                     
robot = MobileRobotWrapper(urdf,[pkg,])
robot.initDisplay(loadModel=True)
gv = robot.viewer.gui

NQ,NV = robot.nq,robot.nv

q = robot.rand()
vq = rand(NV)

robot.display(q)

Remember that the configuration q is living on a manifold, i.e. any q is not an acceptable configuration. Here the constraint is that $q[2]^2+q[3]^2$ should be 1. Then integration should be done carrefully.

In [29]:
import time 
dt = 1e-2
for t in range(1000):
    q = robot.integrate(q,vq*dt)
    robot.display(q)
    time.sleep(dt/10)

The robot is mobile, hence the camera view in Gepetto Viewer is not always centered. Hit space in gepetto Viewer to center the camera.

## Inverse kinematics for one task: position the end effector
The first task will be concerned with the end effector. First define a goal placement.

In [30]:
def place(name,M):
    robot.viewer.gui.applyConfiguration(name,se3ToXYZQUAT(M))
    robot.viewer.gui.refresh()

def Rquat(x,y,z,w): 
    q = pinocchio.Quaternion(x,y,z,w)
    q.normalize()
    return q.matrix()

Mgoal = pinocchio.SE3(Rquat(0.4,0.02, -0.5,0.7),
                       np.matrix([.2,-.4,.7]).T)
gv.addXYZaxis('world/framegoal',[1.,0.,0.,1.],.015,.4) # framecolor, width, length 
place('world/framegoal',Mgoal)


The current placement of the tool at configuration $q$ is available as follows:

In [31]:
IDX_TOOL  = 24
IDX_BASIS = 23
pinocchio.forwardKinematics(robot.model,robot.data,q)   # Compute joint placements
pinocchio.updateFramePlacements(robot.model,robot.data)      # Also compute operational frame placements
Mtool = robot.data.oMf[IDX_TOOL]                  # Get placement from world frame o to frame f oMf
print(Mtool)

  R =
 0.0653317  -0.988231  -0.138316
  0.806045 -0.0294476   0.591121
 -0.588238  -0.150108   0.794635
  p =  2.67198  1.88385 0.537404



The desired velocity of the tool in tool frame is given by the log:

In [32]:
nu = pinocchio.log(Mtool.inverse()*Mgoal).vector
print(nu.T)

[[-1.87453814  2.69019902  1.76324842 -0.76824993 -1.44211519  0.1765847 ]]


The tool Jacobian, also in tool frame, is available as follows:

In [33]:
LOCAL,WORLD = pinocchio.ReferenceFrame.LOCAL,pinocchio.ReferenceFrame.WORLD
J = pinocchio.frameJacobian(robot.model,robot.data,q,IDX_TOOL,LOCAL)
print(J[:,:4])
print(J[:,4:])

[[ 0.78448649  0.19636035 -0.13355859 -0.13355859]
 [-0.34451398  0.92670268 -0.07651796 -0.07651796]
 [ 0.51564626  0.32041342 -0.11332267 -0.11332267]
 [ 0.          0.         -0.58823767 -0.58823767]
 [-0.         -0.         -0.15010752 -0.15010752]
 [ 0.          0.          0.79463462  0.79463462]]
[[-1.41032805e-01 -3.29531402e-02  9.20502131e-02 -8.00000000e-02
   0.00000000e+00]
 [-3.59889992e-02 -8.40904028e-03  2.34895353e-02  1.01481323e-16
   0.00000000e+00]
 [-5.94542966e-02  3.50647071e-01 -1.97806613e-02 -1.74448130e-16
   0.00000000e+00]
 [-2.47258266e-01 -2.47258266e-01 -2.47258266e-01  1.65492620e-15
   0.00000000e+00]
 [ 9.68949612e-01  9.68949612e-01  9.68949612e-01  4.16333634e-16
   0.00000000e+00]
 [-2.77555756e-17 -2.77555756e-17 -2.77555756e-17  1.00000000e+00
   0.00000000e+00]]


Pseudoinverse operator is available in *numpy.linalg* toolbox.

In [34]:
from numpy.linalg import pinv

We provide below the template of the end-effector servo. Complete the template to have the end effector reach the proper placement. Also see the tp3/template.py if you prefer to edit a standalone file.

In [36]:
place('world/framegoal',Mgoal)
place('world/yaxis',pinocchio.SE3(rotate('x',np.pi/2),
                            np.matrix([0,0,.1]).T))

# Define robot initial configuration
q  = robot.rand()
q[:2] = 0  # Basis at the center of the world.

# Loop on an inverse kinematics for 200 iterations.
for i in range(200):   # Integrate over 1 second of robot life
      pinocchio.forwardKinematics(robot.model,robot.data,q)     # Compute joint placements
      pinocchio.updateFramePlacements(robot.model,robot.data)   # Also compute operational frame placements
      Mtool = robot.data.oMf[IDX_TOOL]                          # Get placement from world frame o to frame f oMf
      J  = pinocchio.frameJacobian(robot.model,robot.data,q,IDX_TOOL,LOCAL) # Get corresponding jacobian
      ### ... YOUR CODE HERE
      vq    = rand(NV)   #   .... REPLACE THIS LINE BY YOUR CODE ...
      ### ... END OF YOUR CODE HERE
      q = robot.integrate(q,vq*dt)
      robot.display(q)
      time.sleep(dt)


In [37]:
place('world/framegoal',Mgoal)

# Define robot initial configuration
q  = robot.rand()
q[:2] = 0  # Basis at the center of the world.
Kp= 1.

# Loop on an inverse kinematics for 200 iterations.
for i in range(2000):   # Integrate over 1 second of robot life
      pinocchio.forwardKinematics(robot.model,robot.data,q)     # Compute joint placements
      pinocchio.updateFramePlacements(robot.model,robot.data)   # Also compute operational frame placements
      Mtool = robot.data.oMf[IDX_TOOL]                          # Get placement from world frame o to frame f oMf
      J1  = pinocchio.frameJacobian(robot.model,robot.data,q,IDX_TOOL,LOCAL) # Get corresponding jacobian
      nu1   = Kp*pinocchio.log(Mtool.inverse()*Mgoal).vector     # Compute needed displacement

      vq    = pinv(J1)*nu1
      q = robot.integrate(q,vq*dt)
      robot.display(q)
      time.sleep(dt)


## Position the basis on the line
A line displaying "x=0" is also displayed in Gepetto viewer.
Next step is to servo the front of the basis on this line.

Similarly, the distance of the basis frame to the line, with corresponding jacobian, are:

In [9]:
robot.viewer.gui.addCylinder('world/yaxis',.01,20,[0.1,0.1,0.1,1.])
place('world/yaxis',pinocchio.SE3(rotate('x',np.pi/2),
                            np.matrix([0,0,.1]).T))

Mbasis = robot.data.oMf[IDX_BASIS]
error = Mbasis.translation[0]
J  = pinocchio.frameJacobian(robot.model,robot.data,q,IDX_BASIS,LOCAL)[0,:]


Implement a second loop to servo the basis on the line. It becomes interesting when both tasks are performed together. We can do that simply by summing both tasks. For that, the numpy method +hstack+ can be used to make a single error vector stacking the errors of tool and basis tasks, and similarly for the jacobians. Use the numpy method np.stack([a,b]) with a and b two vectors.

However, it is stronger to move the basis only in the null space of the basis.
The null space projector of +J1+ can be computed using the pseudoinverse.
Following the control law performing task 1 and task 2 in the null space of task 1 is:
$$vq_1 = J_1^+ v_1^*$$
$$P_1 = I_9 - J_1^+ J_1$$
$$vq_2 = vq_1 + (J_2 P_1)^+ ( v_2^* - J_2 vq_1)$$


## Homework
Implement the control loop that sevos the two tasks and submit it through the web application.